# Drop Delta Tables

Dropping Delta Tables is different depending on if the table is managed or unmanaged.  It's important to understand the difference.

## Unmanaged tables

In [6]:
import pyspark
from delta import *
from pyspark.sql.types import *

In [7]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [8]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/powers/opt/miniconda3/envs/mr-delta/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/powers/.ivy2/cache
The jars for the packages stored in: /Users/powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a1addab-6f7c-4706-82d6-10ef0d01616b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 316ms :: artifacts dl 20ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [9]:
cSchema = StructType([StructField("items", StringType())\
                      ,StructField("number", IntegerType())])

test_list = [['furniture', 1], ['games', 3]]

df = spark.createDataFrame(test_list,schema=cSchema)

## Remove files

In [10]:
df.write.format("delta").mode("overwrite").save("tmp/test_table")

In [13]:
!tree tmp/test_table 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/mr-delta/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/mr-delta/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/powers/opt/miniconda3/envs/mr-delta/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


tmp/test_table
├── _delta_log
│   └── 00000000000000000000.json
├── part-00000-00f7a436-6d1c-40fb-9b89-a358e7ed3a3a-c000.snappy.parquet
├── part-00003-6f09df03-ae51-4c07-a565-a0a1738ee354-c000.snappy.parquet
└── part-00007-890d5137-1867-44f4-97c6-cef599392dce-c000.snappy.parquet

1 directory, 4 files


In [14]:
!rm -rf tmp/

In [15]:
!tree tmp/test_table 

tmp/test_table  [error opening dir]

0 directories, 0 files


## Remove managed table

In [16]:
df.write.saveAsTable("your_managed_table")

In [18]:
spark.sql("select * from your_managed_table").show()

+---------+------+
|    items|number|
+---------+------+
|furniture|     1|
|    games|     3|
+---------+------+



In [19]:
!tree spark-warehouse

spark-warehouse
└── your_managed_table
    ├── _SUCCESS
    ├── part-00000-21f03761-1492-47eb-a6d6-d064a21a89d3-c000.snappy.parquet
    ├── part-00003-21f03761-1492-47eb-a6d6-d064a21a89d3-c000.snappy.parquet
    └── part-00007-21f03761-1492-47eb-a6d6-d064a21a89d3-c000.snappy.parquet

1 directory, 4 files


In [21]:
spark.sql("show tables").show()

+---------+------------------+-----------+
|namespace|         tableName|isTemporary|
+---------+------------------+-----------+
|  default|your_managed_table|      false|
+---------+------------------+-----------+



In [22]:
spark.sql("drop table if exists your_managed_table")

DataFrame[]

In [23]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [27]:
!tree spark-warehouse

spark-warehouse

0 directories, 0 files


## Removed unmanaged table

In [24]:
df.write.option("path", "tmp/unmanaged_data").saveAsTable("your_unmanaged_table")

In [25]:
spark.sql("show tables").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|your_unmanaged_table|      false|
+---------+--------------------+-----------+



In [26]:
!tree tmp/

tmp/
└── unmanaged_data
    ├── _SUCCESS
    ├── part-00000-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet
    ├── part-00003-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet
    └── part-00007-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet

1 directory, 4 files


In [28]:
spark.sql("drop table if exists your_unmanaged_table")

22/07/30 17:45:38 WARN HadoopFSUtils: The directory file:/Users/powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/spark-warehouse/tmp/unmanaged_data was not found. Was it deleted very recently?


DataFrame[]

In [29]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [30]:
!tree tmp/

tmp/
└── unmanaged_data
    ├── _SUCCESS
    ├── part-00000-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet
    ├── part-00003-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet
    └── part-00007-ab42755f-6a9e-4fe9-8b0d-ecd6a0b69388-c000.snappy.parquet

1 directory, 4 files
